In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.gold_person_event_timeline AS
SELECT p.person_gedcom_id
	,pn.given_name
	,pn.surname
	,e.event_id
	,e.event_type
	,ep.role AS event_role
	,ep.data_source AS event_participant_source
	,e.event_date_raw
	,e.event_date_type
	,e.event_date_parsed
	,e.event_year_parsed
	,e.event_place
	,e.event_country_inferred
	,CASE 
		WHEN birth_date IS NULL
			OR e.event_date_parsed IS NULL
			THEN NULL
				-- exclude BEFORE / AFTER
		WHEN birth_date_type IN (
				'BEF'
				,'AFT'
				)
			OR event_date_type IN (
				'BEF'
				,'AFT'
				)
			THEN NULL
		ELSE year(event_date_parsed) - year(birth_date) - CASE 
				WHEN date_format(event_date_parsed, 'MM-dd') < date_format(birth_date, 'MM-dd')
					THEN 1
				ELSE 0
				END
		END AS age_years
	,CASE 
		WHEN age_years IS NULL
			THEN NULL
		WHEN birth_date_type = 'EXACT'
			AND event_date_type = 'EXACT'
			THEN true
		ELSE false
		END AS age_years_exact_flag
	,CASE 
		WHEN age_years IS NULL
			THEN NULL
		WHEN age_years_exact_flag = true
			THEN NULL
		ELSE CONCAT (
				'Age estimated from '
				,birth_date_type
				,' birth date and '
				,event_date_type
				,' event date'
				)
		END AS age_note
	,row_number() OVER (
		PARTITION BY p.person_gedcom_id
		,e.event_type
		,ep.role ORDER BY e.event_date_parsed
		) AS event_ordinal_within_type_and_role
	,row_number() OVER (
		PARTITION BY p.person_gedcom_id ORDER BY e.event_date_parsed
			,e.event_type
		) AS event_sequence_by_date
	,
	--Calculate days since previous event of same type/role, ignore NULLs and BEF/AFT dates
	CASE 
		WHEN e.event_date_parsed IS NULL
			THEN NULL
		WHEN event_date_type IN (
				'BEF'
				,'AFT'
				)
			THEN NULL
		WHEN lag(e.event_date_parsed) OVER (
				PARTITION BY p.person_gedcom_id
				,e.event_type
				,ep.role ORDER BY e.event_date_parsed
				) IS NULL
			THEN NULL
		WHEN lag(event_date_type) OVER (
				PARTITION BY p.person_gedcom_id
				,e.event_type
				,ep.role ORDER BY e.event_date_parsed
				) IN (
				'BEF'
				,'AFT'
				)
			THEN NULL
		ELSE datediff(e.event_date_parsed, lag(e.event_date_parsed) OVER (
					PARTITION BY p.person_gedcom_id
					,e.event_type
					,ep.role ORDER BY e.event_date_parsed
					))
		END AS days_since_previous_same_event
	,CASE 
		WHEN days_since_previous_same_event IS NULL
			THEN 'UNKNOWN'
		WHEN days_since_previous_same_event = 0
			THEN 'SAME_DAY'
		WHEN days_since_previous_same_event <= 7
			THEN 'DAYS'
		WHEN days_since_previous_same_event <= 30
			THEN 'WEEKS'
		WHEN days_since_previous_same_event <= 365
			THEN 'MONTHS'
		WHEN days_since_previous_same_event <= 3650
			THEN 'YEARS'
		ELSE 'DECADE_PLUS'
		END AS same_event_gap_bucket
	,
	-- Same calcs for any event
	CASE 
		WHEN e.event_date_parsed IS NULL
			THEN NULL
		WHEN event_date_type IN (
				'BEF'
				,'AFT'
				)
			THEN NULL
		WHEN lag(e.event_date_parsed) OVER (
				PARTITION BY p.person_gedcom_id ORDER BY e.event_date_parsed
				) IS NULL
			THEN NULL
		WHEN lag(event_date_type) OVER (
				PARTITION BY p.person_gedcom_id ORDER BY e.event_date_parsed
				) IN (
				'BEF'
				,'AFT'
				)
			THEN NULL
		ELSE datediff(e.event_date_parsed, lag(e.event_date_parsed) OVER (
					PARTITION BY p.person_gedcom_id ORDER BY e.event_date_parsed
					))
		END AS days_since_previous_any_event
	,CASE 
		WHEN days_since_previous_any_event IS NULL
			THEN 'UNKNOWN'
		WHEN days_since_previous_any_event = 0
			THEN 'SAME_DAY'
		WHEN days_since_previous_any_event <= 7
			THEN 'DAYS'
		WHEN days_since_previous_any_event <= 30
			THEN 'WEEKS'
		WHEN days_since_previous_any_event <= 365
			THEN 'MONTHS'
		WHEN days_since_previous_any_event <= 3650
			THEN 'YEARS'
		ELSE 'DECADE_PLUS'
		END AS any_event_gap_bucket
FROM genealogy.gold_person_life p
JOIN genealogy.silver_person_name pn ON pn.person_gedcom_id = p.person_gedcom_id
JOIN genealogy.silver_event_participant ep ON p.person_gedcom_id = ep.person_id
JOIN genealogy.gold_event e ON ep.event_id = e.event_id
